<a href="https://colab.research.google.com/github/Saim-Hassan786/Tracing-With-OpenAI-Agents-SDK/blob/main/Tracing_With_OpenAI_Agents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -Uq openai-agents

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.2/120.2 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.3/129.3 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 717.5/717.5 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 2.6 MB/s eta 0:00:00


In [2]:
import nest_asyncio
nest_asyncio.apply()

In [3]:
from google.colab import userdata
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')

In [4]:
from agents import Agent,Runner,set_default_openai_api,set_default_openai_client
from openai import AsyncOpenAI

external_client = AsyncOpenAI(
    api_key=GOOGLE_API_KEY,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

set_default_openai_client(external_client)
set_default_openai_api("chat_completions")

# The Code Below Is For Tracing With OpenAI API Key And Their Tracing Platform

In [13]:
from agents import trace
async def main():
   agent = Agent(
    name = "Joke Teller Agent",
    instructions="Tell a joke to the user",
    model = "gemini-2.0-flash"
)

   with trace("Joker Tracing"):
    result_1 = await Runner.run(agent, "Tell me a joke")
    result_2 = await Runner.run(agent, f"Rate this joke out of 10 {result_1.final_output}")
    print(result_1.final_output)
    print(result_2.final_output)

In [14]:
import asyncio
asyncio.run(main())

Why don't scientists trust atoms? 

Because they make up everything!

Okay, I'd rate that joke a **7/10**.

Here's why:

*   **It's a classic:** The pun is well-known and relies on the double meaning of "make up." It's a reliable, crowd-pleasing joke.
*   **Simple and easy to understand:** The joke doesn't require a lot of background knowledge or complex thinking. Anyone who has a basic understanding of science and language can get it.
*   **Pun-based humor:** Puns are often enjoyable, as they create a humorous twist on words.
*   **Slightly overused:** Because it's so well-known, some people might have heard it many times before, which reduces its impact.
*   **Clean and inoffensive:** Suitable for all audiences

Overall, it's a good, solid joke that's likely to get at least a chuckle.



# All The Above Generated tracings Can be Viewed And Traced Through OpenAI Platform

# Now To View racing Locally Also Called As Local Tracing

In [20]:
from agents import (
    Agent,
    Runner,
    set_default_openai_client,
    set_default_openai_api,
    set_trace_processors
)

from pprint import pprint
from openai import AsyncOpenAI
from agents.tracing.processor_interface import TracingProcessor

class LocalTracing(TracingProcessor):
  def __init__(self):
    self.traces = []
    self.spans = []

  def on_trace_start(self, trace):
    self.traces.append(trace)
    print(f"Trace Started: {trace.trace_id}")

  def on_trace_end(self, trace):
    print(f"Trace Ended :{trace.export()}")

  def on_span_start(self, span):
    self.spans.append(span)
    print(f"Span Started: {span.span_id}")
    print("Spane Details:")
    pprint(span.export())

  def on_span_end(self, span):
    print(f"Span Ended: {span.span_id}")
    print("Spane Details:")
    pprint(span.export())

  def force_flush(self):
    print("Flushing traces and spans")

  def shutdown(self):
    print("=======Shuting Down========")
    print("Collected Traces")
    for trace in self.traces:
      print(trace.export())
    print("Collected Spans")
    for span in self.spans:
      print(span.export())

external_client = AsyncOpenAI(
    api_key=GOOGLE_API_KEY,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

set_default_openai_client(
    client=external_client,
    use_for_tracing=True
)

set_default_openai_api("chat_completions")

local_tracing = LocalTracing()
set_trace_processors([local_tracing])


from agents import trace
async def main_1():
   agent = Agent(
    name = "Joke Teller Agent",
    instructions="Tell a joke to the user",
    model = "gemini-2.0-flash"
)

   with trace("Joker Tracing"):
    result_1 = await Runner.run(agent, "Tell me a joke")
    result_2 = await Runner.run(agent, f"Rate this joke out of 10 {result_1.final_output}")
    print(result_1.final_output)
    print(result_2.final_output)

In [21]:
asyncio.run(main_1())

Trace Started: trace_04d70b6ce85b477ea557338afc31422b
Span Started: span_04c589b47f0c4eb19034c117
Spane Details:
{'ended_at': None,
 'error': None,
 'id': 'span_04c589b47f0c4eb19034c117',
 'object': 'trace.span',
 'parent_id': None,
 'span_data': {'handoffs': [],
               'name': 'Joke Teller Agent',
               'output_type': 'str',
               'tools': None,
               'type': 'agent'},
 'started_at': '2025-05-22T15:20:05.546980+00:00',
 'trace_id': 'trace_04d70b6ce85b477ea557338afc31422b'}
Span Started: span_dcedfd3ad8f249d5bb93e5bf
Spane Details:
{'ended_at': None,
 'error': None,
 'id': 'span_dcedfd3ad8f249d5bb93e5bf',
 'object': 'trace.span',
 'parent_id': 'span_04c589b47f0c4eb19034c117',
 'span_data': {'input': None,
               'model': 'gemini-2.0-flash',
               'model_config': {'base_url': 'https://generativelanguage.googleapis.com/v1beta/openai/',
                                'extra_body': None,
                                'extra_headers': N

# To Integrate A 3rd Party Interface To View Tracing Like AgentOPS

In [22]:
!pip install -Uq agentops

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.3/198.3 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.5/290.5 kB 20.0 MB/s eta 0:00:00


In [23]:
from google.colab import userdata
AGENTOPS_API_KEY = userdata.get("AGENTOPS_API_KEY")

In [24]:
import agentops
agentops.init(AGENTOPS_API_KEY)

🖇 AgentOps: Session Replay: https://app.agentops.ai/sessions?trace_id=65e480f46609ead333b085792908cb70


In [25]:
async def main_2():
   agent = Agent(
    name = "Joke Teller Agent",
    instructions="Tell a joke to the user",
    model = "gemini-2.0-flash"
)

   with trace("Joker Tracing"):
    result_1 = await Runner.run(agent, "Tell me a joke")
    result_2 = await Runner.run(agent, f"Rate this joke out of 10 {result_1.final_output}")
    print(result_1.final_output)
    print(result_2.final_output)

asyncio.run(main_2())

Why don't scientists trust atoms?

Because they make up everything!

I'd rate that joke a solid 7/10. 

Here's why:

*   **It's a classic:** The pun is well-known and predictable, but that also makes it accessible and easy to understand.
*   **It's clean:** No offensive content or complicated setup.
*   **It's clever (in a simple way):** The wordplay on "make up" is the key to the humor.
*   **It's not laugh-out-loud funny:** While it's a good joke, it's unlikely to cause strong laughter. More of a chuckle or a smile.

Overall, it's a reliable, pun-based joke that's good for a quick, lighthearted moment.

